In [1]:
from openai import OpenAI
import json

## Creating docs

In [20]:
with open('../documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [21]:
documents = []

for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

## Building prompt templates and response

In [2]:
client = OpenAI()

In [3]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teacher assistant. Answer the question based on the context.
        Use only the facts from the context. If relevant information is missing, say you do not know.
        Question:
        {query}
        Context:
        {context}
        """.strip()
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    return prompt_template.format(query=query, context=context).strip()

In [4]:
def llm(client, prompt, model='gpt-4.1-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt }]
    )
    return response.choices[0].message.content

## Preparing search engine

In [8]:
from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
es_client.info()

ObjectApiResponse({'name': 'e2d01f351069', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hTAMenX5Q5qZo2PhzkoAaA', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
index_settings ={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:05<00:00, 184.55it/s]


In [27]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 19, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 334, 'relation': 'eq'}, 'max_score': 31.973522, 'hits': [{'_index': 'course-questions', '_id': 'MjpzQZcBViTmiCa5Sxs9', '_score': 31.973522, '_source': {'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:', 'section': 'Course Management Form for Homeworks', 'question': 'How to run a dbt-core project as an Airflow Task Grou

In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    responses_list = []
    for hit in response['hits']['hits']:
        responses_list.append(hit['_source'])
    return responses_list

In [12]:
def rag(query, verbose_search= False, verbose_prompt = False):
    search_results = elastic_search(q)
    if verbose_search:
        print(search_results)
    prompt = build_prompt(q,search_results)
    if verbose_prompt:
        print(prompt)
    message = llm(client, prompt)
    return(message)

In [13]:
q = "How do copy a file to a Docker container?"

In [23]:
rag(q, verbose_search=True)

[{'text': 'Install the astronomer-cosmos package as a dependency. (see Terraform example).\nMake a new folder, dbt/, inside the dags/ folder of your Composer GCP bucket and copy paste your dbt-core project there. (see example)\nEnsure your profiles.yml is configured to authenticate with a service account key. (see BigQuery example)\nCreate a new DAG using the DbtTaskGroup class and a ProfileConfig specifying a profiles_yml_filepath that points to the location of your JSON key file. (see example)\nYour dbt lineage graph should now appear as tasks inside a task group like this:', 'section': 'Course Management Form for Homeworks', 'question': 'How to run a dbt-core project as an Airflow Task Group on Google Cloud Composer using a service account JSON key', 'course': 'data-engineering-zoomcamp'}, {'text': "There are 3 Zoom Camps in a year, as of 2024. However, they are for separate courses:\nData-Engineering (Jan - Apr)\nMLOps (May - Aug)\nMachine Learning (Sep - Jan)\nThere's only one Dat

'The provided context does not contain information on how to execute a command on a Kubernetes pod. Therefore, I do not know.'